In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, MultiHeadAttention, LayerNormalization, Dense, Dropout
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Load dataset
df = pd.read_csv('/kaggle/input/pali-to-english/output_file.csv')

# Separate the input and target text
input_texts = df['input_text'].values
target_texts = df['target_text'].values

# Tokenization and padding (you can adjust max_length based on your data)
tokenizer_pali = tf.keras.preprocessing.text.Tokenizer()
tokenizer_english = tf.keras.preprocessing.text.Tokenizer()

tokenizer_pali.fit_on_texts(input_texts)
tokenizer_english.fit_on_texts(target_texts)

input_sequences = tokenizer_pali.texts_to_sequences(input_texts)
target_sequences = tokenizer_english.texts_to_sequences(target_texts)

max_len_input = max([len(seq) for seq in input_sequences])
max_len_target = max([len(seq) for seq in target_sequences])

input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_len_input, padding='post')
target_sequences = tf.keras.preprocessing.sequence.pad_sequences(target_sequences, maxlen=max_len_target, padding='post')

# Vocabulary sizes
vocab_size_pali = len(tokenizer_pali.word_index) + 1
vocab_size_english = len(tokenizer_english.word_index) + 1

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(input_sequences, target_sequences[:, 1:], test_size=0.2, random_state=42)
target_sequences_input = target_sequences[:, :-1]


In [3]:
class Transformer(Model):
    def __init__(self, vocab_size_input, vocab_size_target, embed_dim, num_heads, ff_dim, num_layers, dropout_rate=0.1):
        super(Transformer, self).__init__()
        self.embedding_input = Embedding(vocab_size_input, embed_dim)
        self.embedding_target = Embedding(vocab_size_target, embed_dim)

        self.encoder_layers = [
            [MultiHeadAttention(num_heads, embed_dim // num_heads),
             LayerNormalization(epsilon=1e-6),
             Dense(ff_dim, activation='relu'),
             Dense(embed_dim),
             Dropout(dropout_rate)]
            for _ in range(num_layers)
        ]

        self.decoder_layers = [
            [MultiHeadAttention(num_heads, embed_dim // num_heads),
             MultiHeadAttention(num_heads, embed_dim // num_heads),
             LayerNormalization(epsilon=1e-6),
             Dense(ff_dim, activation='relu'),
             Dense(embed_dim),
             Dropout(dropout_rate)]
            for _ in range(num_layers)
        ]

        self.final_layer = Dense(vocab_size_target)

    def call(self, inputs, training=False):
        input_seq, target_seq = inputs
        input_embedding = self.embedding_input(input_seq)
        target_embedding = self.embedding_target(target_seq)

        x = input_embedding
        for mha, ln, ff1, ff2, drop in self.encoder_layers:
            attn_output = mha(x, x)
            x = ln(x + attn_output)
            ffn_output = ff2(ff1(x))
            x = drop(x + ffn_output, training=training)

        y = target_embedding
        for mha1, mha2, ln, ff1, ff2, drop in self.decoder_layers:
            attn_output1 = mha1(y, y)
            attn_output2 = mha2(attn_output1, x)
            y = ln(y + attn_output2)
            ffn_output = ff2(ff1(y))
            y = drop(y + ffn_output, training=training)

        return self.final_layer(y)


In [4]:
# Model parameters
embed_dim = 256  # Embedding size
num_heads = 8    # Number of attention heads
ff_dim = 512     # Feed-forward network size
num_layers = 4   # Number of encoder/decoder layers

transformer = Transformer(vocab_size_pali, vocab_size_english, embed_dim, num_heads, ff_dim, num_layers)

# Compile the model with accuracy metric
transformer.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

# Train the model for 10 epochs with a batch size of 124
transformer.fit([X_train, target_sequences_input[:len(X_train)]],
                y_train,
                batch_size=16,  # or 32
                epochs=10)



Epoch 1/10


I0000 00:00:1728976866.833967      82 service.cc:145] XLA service 0x793ed4003e70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728976866.834020      82 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1728976866.834024      82 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
W0000 00:00:1728976868.619541      82 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1728976887.367542     110 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_70', 1076 bytes spill stores, 1076 bytes spill loads

I0000 00:00:1728976891.008395     108 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_457', 332 bytes spill stores, 312 bytes spill loads

I0000 00:00:172897

6607/6608 ━━━━━━━━━━━━━━━━━━━━ 0s 618ms/step - accuracy: 0.9753 - loss: 0.3292

I0000 00:00:1728981036.880356     224 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_475', 12 bytes spill stores, 12 bytes spill loads

I0000 00:00:1728981043.701647     224 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_467', 1764 bytes spill stores, 1764 bytes spill loads

I0000 00:00:1728981046.704396     222 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_452', 1764 bytes spill stores, 1764 bytes spill loads

I0000 00:00:1728981058.585968     222 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_446', 100 bytes spill stores, 100 bytes spill loads

I0000 00:00:1728981063.911056     224 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_458', 1764 bytes spill stores, 1764 bytes spill loads

I0000 00:00:172898106

6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4265s 631ms/step - accuracy: 0.9753 - loss: 0.3292
Epoch 2/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4033s 610ms/step - accuracy: 0.9766 - loss: 0.2739
Epoch 3/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4015s 608ms/step - accuracy: 0.9767 - loss: 0.2723
Epoch 4/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4012s 607ms/step - accuracy: 0.9767 - loss: 0.2719
Epoch 5/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4012s 607ms/step - accuracy: 0.9767 - loss: 0.2719
Epoch 6/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4011s 607ms/step - accuracy: 0.9767 - loss: 0.2713
Epoch 7/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4010s 607ms/step - accuracy: 0.9767 - loss: 0.2718
Epoch 8/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4009s 607ms/step - accuracy: 0.9766 - loss: 0.2717
Epoch 9/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 4002s 606ms/step - accuracy: 0.9767 - loss: 0.2714
Epoch 10/10
6608/6608 ━━━━━━━━━━━━━━━━━━━━ 3996s 605ms/step - accuracy: 0.9767 - loss: 0.2709


In [5]:
# Evaluate on the validation set
val_loss, val_accuracy = transformer.evaluate([X_val, target_sequences_input[len(X_train):]], y_val)

print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


W0000 00:00:1729017211.134732      81 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1729017213.648433     447 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_80', 12 bytes spill stores, 12 bytes spill loads

I0000 00:00:1729017214.245391     448 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_78', 332 bytes spill stores, 312 bytes spill loads

I0000 00:00:1729017214.722991     445 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_78', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1729017217.725884     446 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_88', 12 bytes spill stores, 12 bytes spill loads



825/826 ━━━━━━━━━━━━━━━━━━━━ 0s 396ms/step - accuracy: 0.9768 - loss: 0.2701

W0000 00:00:1729017556.263868      79 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1729017557.064799     476 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_78', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1729017559.293940     477 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_78', 332 bytes spill stores, 312 bytes spill loads



826/826 ━━━━━━━━━━━━━━━━━━━━ 364s 417ms/step - accuracy: 0.9768 - loss: 0.2701
Validation Loss: 0.27197015285491943
Validation Accuracy: 0.9765858054161072
